# Merge Questionnaire Data

In [109]:
import json
import re
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

from fau_colors import cmaps
import biopsykit as bp

%load_ext autoreload
%autoreload 2
%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [110]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

palette

[(0.0, 0.1843137254901961, 0.4235294117647059),
 (0.4666666666666667, 0.6235294117647059, 0.7098039215686275),
 (1.0, 0.7215686274509804, 0.10980392156862745),
 (0.0, 0.6392156862745098, 0.8784313725490196),
 (0.2627450980392157, 0.6901960784313725, 0.16470588235294117),
 (0.7843137254901961, 0.06274509803921569, 0.1803921568627451)]

In [111]:
deploy_type = "local"

In [112]:
config_dict = json.load(Path("../config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])
base_path

PosixPath('/Volumes/luca_ssd/Study_Data/2022_05_AP01_Macro')

In [113]:
code_mapping = pd.read_csv(base_path.joinpath("data_tabular/extras/code_to_number_mapping.csv"))
code_mapping = code_mapping.set_index("Code")
code_mapping.head()

,subject
Code,
BG05W,VP_01
HZ03B,VP_02
NW15N,VP_03
KS08F,VP_04
FA01B,VP_05


## Load Questionnaire Data and Map Code to VP-ID

In [114]:
quest_data = pd.read_excel(base_path.joinpath("data_tabular/questionnaires/raw/unipark_screening.xlsx"))
quest_data = quest_data.rename(columns={"VPN_Kennung": "Code"})
quest_data = quest_data.set_index("Code")

quest_data = quest_data.join(code_mapping)

quest_data = quest_data.dropna(subset=["subject"]).set_index("subject")

quest_data

,lfdn,external_lfdn,tester,dispcode,lastpage,quality,duration,Einverstaendnis_1,Einverstaendnis_2,Geschlecht,...,rts7050127,rts7050128,rts7050129,rts7050130,rts7050131,rts7050132,rts7050133,rts7050134,rts7070232,rts7071496
subject,,,,,,,,,,,,,,,,,,,,,
VP_09,130,0,0,31,7033748,-77,244,1,1,1,...,0,0,0,0,0,0,0,0,0,0
VP_09,131,0,0,31,7033773,-77,394,1,1,1,...,0,0,0,0,0,0,0,0,0,0
VP_09,158,0,0,31,7033773,-77,617,1,1,1,...,0,0,0,0,0,0,0,0,0,0
VP_09,306,0,0,31,7033731,-77,1136,1,1,1,...,321,366,417,448,549,653,720,893,1116,1130
VP_25,554,0,0,31,7033731,-77,1154,1,1,2,...,419,504,605,666,768,877,955,1083,1132,1147
VP_27,542,0,0,31,7033754,-77,570,1,1,2,...,0,0,0,0,0,0,0,0,0,0
VP_27,543,0,0,31,7033731,-77,2372,1,1,2,...,652,814,903,1103,1411,1558,1668,2038,2338,2367
VP_15,113,0,0,31,7033731,-77,2253,1,1,1,...,906,1004,1092,1186,1381,1560,1673,2121,2226,2243
VP_01,57,0,0,31,7033731,-77,2225,1,1,1,...,1148,1237,1371,1430,1614,1776,1860,2092,2203,2220


In [115]:
# keep last entry for each subject
quest_data = quest_data.groupby("subject").last()

In [116]:
quest_data

,lfdn,external_lfdn,tester,dispcode,lastpage,quality,duration,Einverstaendnis_1,Einverstaendnis_2,Geschlecht,...,rts7050127,rts7050128,rts7050129,rts7050130,rts7050131,rts7050132,rts7050133,rts7050134,rts7070232,rts7071496
subject,,,,,,,,,,,,,,,,,,,,,
VP_01,57,0,0,31,7033731,-77,2225,1,1,1,...,1148,1237,1371,1430,1614,1776,1860,2092,2203,2220
VP_02,129,0,0,31,7033731,-77,1623,1,1,1,...,660,718,810,856,1111,1257,1347,1551,1608,1619
VP_03,135,0,0,31,7033731,-77,2565,1,1,2,...,682,1009,1119,1180,1437,1622,2012,2415,2530,2551
VP_04,177,0,0,31,7033731,-77,2440,1,1,2,...,830,907,1030,1095,1502,1923,2029,2320,2423,2434
VP_05,79,0,0,31,7033731,-77,1716,1,1,2,...,662,722,807,859,1029,1186,1300,1603,1681,1708
VP_06,308,0,0,31,7033731,-77,1999,1,1,2,...,1076,1131,1211,1258,1426,1535,1615,1894,1957,1992
VP_07,328,0,0,31,7033731,-77,2457,1,1,1,...,1027,1181,1381,1465,1736,1909,2014,2329,2432,2450
VP_08,382,0,0,31,7033731,-77,1962,1,1,2,...,695,763,852,906,1133,1312,1426,1828,1942,1955
VP_09,306,0,0,31,7033731,-77,1136,1,1,1,...,321,366,417,448,549,653,720,893,1116,1130


In [117]:
# load renaming json
renaming_dict = json.load(Path("renaming.json").open(encoding="utf-8"))

renaming_dict

{'Geschlecht': 'Gender',
 'Alter': 'Age',
 'Geburtsdatum': 'Birth_Date',
 'Familienstand': 'Marital',
 'Bildungsabschluss': 'Education',
 'Beschaeftigungsverhaeltnis': 'Profession',
 'Beschaeftigungsverhaeltnis_Sonstige': 'Profession_Other',
 'Ethnische_Herkunft': 'Ethnicity',
 'Studiengang': 'Course_Study',
 'Berufsfeld': 'Professional_Field',
 'Muttersprache': 'Native_Language',
 'Koerpergroesse': 'Height',
 'Koerpergewicht': 'Weight',
 'Haendigkeit': 'Handedness',
 'Kontrazeptiva_Keine': 'Contraceptives_None',
 'Kontrazeptiva_Pille': 'Contraceptives_Pill',
 'Kontrazeptiva_hormonfrei': 'Contraceptives_Hormone_Free',
 'Kontrazeptiva_hormonell_andere': 'Contraceptives_Hormone_Other',
 'Menstruation_regelm_Zyklus': 'Menstruation_Regular_Cycle',
 'Menstruation_unregelm_Zyklus_Grund': 'Menstruation_Unregular_Cycle_Reason',
 'Menstruation_letzte_Regelblutung': 'Menstruation_Last_Period',
 'Zykluslaenge': 'Cycle_Length',
 'Menstruation_Zyklus_Schwankung': 'Menstruation_Cycle_Fluctuations',


In [118]:
quest_data = quest_data.rename(renaming_dict, axis=1)

# get only columns that  are in renaming dict and in the quest_data
columns = list(set(list(renaming_dict.values())) & set(quest_data.columns))

quest_data_filtered = quest_data[columns]

quest_data_filtered.head()

,Disease_Skeleton,Marital,Disease_Eye,Native_Language,Medication_Anti_Inflammatory,Education,Menstruation_Last_Period,Menstruation_Ovulation_Tracking,Menstruation_Cycle_Fluctuations,Date_Ovulation,...,Disease_Liver,Disease_Kidney,Disease_Acute_Symptoms,Profession_Other,Disease_Nervous_System,Disease_Cancer,Disease_Metabolism,Menstruation_Regular_Cycle,Medication_Painkiller,Age
subject,,,,,,,,,,,,,,,,,,,,,
VP_01,2,2,2,1,2,5,2022-01-06,0,4,2022-02-16,...,2,2,2,-99,2,2,2,2,2,28
VP_02,2,1,2,1,2,6,2022-03-06,-77,1,-66,...,2,2,2,-99,2,2,2,1,2,29
VP_03,2,1,2,1,2,6,-66,-77,-77,-66,...,2,2,2,-99,2,2,2,-77,2,23
VP_04,2,1,2,1,2,5,-66,-77,-77,-66,...,2,2,2,-99,2,2,2,-77,2,24
VP_05,2,1,2,1,2,5,-66,-77,-77,-66,...,2,2,2,-99,2,2,2,-77,2,23


In [119]:
# replace all missing values
quest_data_filtered = quest_data_filtered.replace("-66", np.nan)
quest_data_filtered = quest_data_filtered.replace(-77, np.nan)

In [120]:
quest_data_filtered

,Disease_Skeleton,Marital,Disease_Eye,Native_Language,Medication_Anti_Inflammatory,Education,Menstruation_Last_Period,Menstruation_Ovulation_Tracking,Menstruation_Cycle_Fluctuations,Date_Ovulation,...,Disease_Liver,Disease_Kidney,Disease_Acute_Symptoms,Profession_Other,Disease_Nervous_System,Disease_Cancer,Disease_Metabolism,Menstruation_Regular_Cycle,Medication_Painkiller,Age
subject,,,,,,,,,,,,,,,,,,,,,
VP_01,2,2,2,1,2,5,2022-01-06,0.0,4.0,2022-02-16,...,2,2,2,-99,2,2,2,2.0,2,28
VP_02,2,1,2,1,2,6,2022-03-06,NaN,1.0,NaN,...,2,2,2,-99,2,2,2,1.0,2,29
VP_03,2,1,2,1,2,6,NaN,NaN,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,23
VP_04,2,1,2,1,2,5,NaN,NaN,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,24
VP_05,2,1,2,1,2,5,NaN,NaN,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,23
VP_06,2,1,2,1,2,5,NaN,NaN,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,19
VP_07,2,2,2,1,2,4,2022-03-14,1.0,4.0,2022-03-23,...,2,2,2,-99,2,2,2,2.0,2,23
VP_08,2,1,2,1,2,5,NaN,NaN,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,22
VP_09,2,1,2,1,2,5,2022-03-01,NaN,2.0,NaN,...,2,2,2,-99,2,2,2,1.0,2,25


## Add Condition Order

In [121]:
condition_order = pd.read_csv(base_path.joinpath("data_tabular/extras/condition_order.csv"))
condition_order = condition_order.set_index("subject")["condition_order"]
condition_order.head()

# change condtion order to codes
condition_order = condition_order.replace("tsst_first", 1)
condition_order = condition_order.replace("ftsst_first", 2)

quest_data_filtered.insert(1, "Condition_Order", condition_order)

In [122]:
quest_data_filtered

,Disease_Skeleton,Condition_Order,Marital,Disease_Eye,Native_Language,Medication_Anti_Inflammatory,Education,Menstruation_Last_Period,Menstruation_Ovulation_Tracking,Menstruation_Cycle_Fluctuations,...,Disease_Liver,Disease_Kidney,Disease_Acute_Symptoms,Profession_Other,Disease_Nervous_System,Disease_Cancer,Disease_Metabolism,Menstruation_Regular_Cycle,Medication_Painkiller,Age
subject,,,,,,,,,,,,,,,,,,,,,
VP_01,2,1,2,2,1,2,5,2022-01-06,0.0,4.0,...,2,2,2,-99,2,2,2,2.0,2,28
VP_02,2,1,1,2,1,2,6,2022-03-06,NaN,1.0,...,2,2,2,-99,2,2,2,1.0,2,29
VP_03,2,2,1,2,1,2,6,NaN,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,23
VP_04,2,2,1,2,1,2,5,NaN,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,24
VP_05,2,2,1,2,1,2,5,NaN,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,23
VP_06,2,2,1,2,1,2,5,NaN,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,19
VP_07,2,1,2,2,1,2,4,2022-03-14,1.0,4.0,...,2,2,2,-99,2,2,2,2.0,2,23
VP_08,2,1,1,2,1,2,5,NaN,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,22
VP_09,2,1,1,2,1,2,5,2022-03-01,NaN,2.0,...,2,2,2,-99,2,2,2,1.0,2,25


In [89]:
# add to codebook

codebook = pd.read_csv(base_path.joinpath("data_tabular/questionnaires/codebook.csv"))
codebook.set_index("variable", inplace=True)

In [90]:
codebook

,1,2,3,4,5,6,7,8,9,10,11,12,13
variable,,,,,,,,,,,,,
Gender,female,male,other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Education,kein Schulabschluss,Hauptschule,Mittlere Reife,Fachabitur,Abitur,Bachelor,Master/Diplom,Promotion,Habilitation,NaN,NaN,NaN,NaN
Marital,ledig,feste Partnerschaft,verheiratet,getrennt lebend,geschieden,verwitwet,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Profession,Arbeitslos/Arbeitssuchend,Auszubildende/r,Student/in,Hausfrau/mann,Angestellte/r,Beamte/in,Selbstständig,Elternzeit,Beurlaubt,Sabbatjahr,Berufsunfähig/Frührente,Ruhestand,sonstiges
Ethnicity,Asiatisch,Arabisch,Afrikanisch,Indigen,Weiß,Andere,Keine Antwort,NaN,NaN,NaN,NaN,NaN,NaN
Condition_Order,tsst_first,ftsst_first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Handedness,right,left,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
# add row for condition order with nan values
codebook.loc["Condition_Order"] = np.nan

codebook.loc["Condition_Order", "1"] = "tsst_first"
codebook.loc["Condition_Order", "2"] = "ftsst_first"

In [70]:
# export
codebook.to_csv(base_path.joinpath("data_tabular/questionnaires/codebook.csv"))

## Add Handedness

In [123]:
handedness = pd.read_excel(base_path.joinpath("data_tabular/extras/handedness.xlsx"))
handedness = handedness.set_index("subject")["handedness"]

# recode handedness
handedness = handedness.replace("right", 1)
handedness = handedness.replace("left", 2)

quest_data_filtered.insert(1, "Handedness", handedness)
quest_data_filtered.head()

,Disease_Skeleton,Handedness,Condition_Order,Marital,Disease_Eye,Native_Language,Medication_Anti_Inflammatory,Education,Menstruation_Last_Period,Menstruation_Ovulation_Tracking,...,Disease_Liver,Disease_Kidney,Disease_Acute_Symptoms,Profession_Other,Disease_Nervous_System,Disease_Cancer,Disease_Metabolism,Menstruation_Regular_Cycle,Medication_Painkiller,Age
subject,,,,,,,,,,,,,,,,,,,,,
VP_01,2,1,1,2,2,1,2,5,2022-01-06,0.0,...,2,2,2,-99,2,2,2,2.0,2,28
VP_02,2,1,1,1,2,1,2,6,2022-03-06,NaN,...,2,2,2,-99,2,2,2,1.0,2,29
VP_03,2,1,2,1,2,1,2,6,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,23
VP_04,2,2,2,1,2,1,2,5,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,24
VP_05,2,1,2,1,2,1,2,5,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,23


In [124]:
# add to codebook

# add row for handedness with nan values
codebook.loc["Handedness"] = np.nan

codebook.loc["Handedness", "1"] = "right"
codebook.loc["Handedness", "2"] = "left"

# export
codebook.to_csv(base_path.joinpath("data_tabular/questionnaires/codebook.csv"))

In [125]:
quest_data_filtered

,Disease_Skeleton,Handedness,Condition_Order,Marital,Disease_Eye,Native_Language,Medication_Anti_Inflammatory,Education,Menstruation_Last_Period,Menstruation_Ovulation_Tracking,...,Disease_Liver,Disease_Kidney,Disease_Acute_Symptoms,Profession_Other,Disease_Nervous_System,Disease_Cancer,Disease_Metabolism,Menstruation_Regular_Cycle,Medication_Painkiller,Age
subject,,,,,,,,,,,,,,,,,,,,,
VP_01,2,1,1,2,2,1,2,5,2022-01-06,0.0,...,2,2,2,-99,2,2,2,2.0,2,28
VP_02,2,1,1,1,2,1,2,6,2022-03-06,NaN,...,2,2,2,-99,2,2,2,1.0,2,29
VP_03,2,1,2,1,2,1,2,6,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,23
VP_04,2,2,2,1,2,1,2,5,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,24
VP_05,2,1,2,1,2,1,2,5,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,23
VP_06,2,1,2,1,2,1,2,5,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,19
VP_07,2,1,1,2,2,1,2,4,2022-03-14,1.0,...,2,2,2,-99,2,2,2,2.0,2,23
VP_08,2,1,1,1,2,1,2,5,NaN,NaN,...,2,2,2,-99,2,2,2,NaN,2,22
VP_09,2,1,1,1,2,1,2,5,2022-03-01,NaN,...,2,2,2,-99,2,2,2,1.0,2,25


## Export Base

In [127]:
quest_data_filtered.sort_index(axis=1, inplace=True)
quest_data_filtered.to_csv(base_path.joinpath("data_tabular/questionnaires/final/base_data.csv"))

## PASA

In [106]:
pasa_data = pd.read_excel(base_path.joinpath("data_tabular/questionnaires/raw/pasa.xlsx"))
pasa_data = pasa_data.set_index(["subject", "condition"])

# add "PASA_" to column names 
pasa_data.columns = "PASA_" + pasa_data.columns.astype(str).str.zfill(2)
pasa_data = pasa_data.unstack("condition")

pasa_data = pasa_data.sort_index(axis=1, level="condition")

# flatten multiindex
pasa_data.columns = ["_".join(col).strip() for col in pasa_data.columns.values]

pasa_data.head()


,PASA_01_ftsst,PASA_02_ftsst,PASA_03_ftsst,PASA_04_ftsst,PASA_05_ftsst,PASA_06_ftsst,PASA_07_ftsst,PASA_08_ftsst,PASA_09_ftsst,PASA_10_ftsst,...,PASA_07_tsst,PASA_08_tsst,PASA_09_tsst,PASA_10_tsst,PASA_11_tsst,PASA_12_tsst,PASA_13_tsst,PASA_14_tsst,PASA_15_tsst,PASA_16_tsst
subject,,,,,,,,,,,,,,,,,,,,,
VP_01,4,5,5,5,2,2,0,4.0,2,3,...,3,4.0,0,1,2,3,4.0,4.0,1.0,4.0
VP_02,4,4,2,3,1,2,3,2.0,3,3,...,4,1.0,2,1,0,3,4.0,4.0,2.0,3.0
VP_03,5,3,4,3,1,2,1,3.0,5,3,...,0,4.0,4,2,3,3,2.0,3.0,3.0,4.0
VP_04,4,3,4,1,0,4,0,4.0,5,5,...,0,3.0,3,2,4,1,1.0,4.0,3.0,3.0
VP_05,2,2,1,3,4,1,3,3.0,2,2,...,3,3.0,2,2,2,2,3.0,3.0,3.0,2.0


In [107]:
pasa_data.to_csv(base_path.joinpath("data_tabular/questionnaires/final/pasa.csv"))